In [0]:
## Import libraries
from pyspark.ml.classification import LogisticRegression # Logistic Regerssion
from pyspark.ml.evaluation import MulticlassClassificationEvaluator # Evaluate our model
from pyspark.ml.regression import RandomForestRegressor
# Feature Engineering
from pyspark.ml.feature import VectorAssembler, StringIndexer, Tokenizer, StopWordsRemover, CountVectorizer

from pyspark.ml import Pipeline # Pipeline - similar to SKlearn
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder # CV and 

import mlflow
from pyspark.ml.feature import OneHotEncoder

In [0]:
pip install tensorflow-cpu==2.4.*

Python interpreter will be restarted.
Collecting tensorflow-cpu==2.4.*
 Using cached tensorflow_cpu-2.4.3-cp38-cp38-manylinux2010_x86_64.whl (144.4 MB)
Collecting h5py~=2.10.0
 Using cached h5py-2.10.0-cp38-cp38-manylinux1_x86_64.whl (2.9 MB)
Requirement already satisfied: protobuf>=3.9.2 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (3.17.2)
Requirement already satisfied: typing-extensions~=3.7.4 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (3.7.4.3)
Requirement already satisfied: wrapt~=1.12.1 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.12.1)
Requirement already satisfied: absl-py~=0.10 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (0.11.0)
Requirement already satisfied: tensorboard~=2.4 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (2.5.0)
Requirement already satisfied: google-pasta~=0.2 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (0.2.0)
Collecting tensorflow-estimator<2.5.0,>=2.4.0
 Using cached tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
Collecting grpcio~=1.32.0
 Using cached grpcio-1.32.0-cp38-cp38-manylinux2014_x86_64.whl (3.8 MB)
Requirement already satisfied: flatbuffers~=1.12.0 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.12)
Requirement already satisfied: keras-preprocessing~=1.1.2 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.1.2)
Requirement already satisfied: numpy~=1.19.2 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.19.2)
Collecting gast==0.3.3
 Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
Requirement already satisfied: opt-einsum~=3.3.0 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (3.3.0)
Requirement already satisfied: six~=1.15.0 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.15.0)
Requirement already satisfied: wheel~=0.35 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (0.36.2)
Requirement already satisfied: termcolor~=1.1.0 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.1.0)
Requirement already satisfied: astunparse~=1.6.3 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.6.3)
Requirement already satisfied: google-auth<2,>=1.6.3 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (1.22.1)
Requirement already satisfied: werkzeug>=0.11.15 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (1.0.1)
Requirement already satisfied: requests<3,>=2.21.0 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (2.25.1)
Requirement already satisfied: google-auth-oauthlib<0.5,>=0.4.1 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (0.4.2)
Requirement already satisfied: setuptools>=41.0.0 in /usr/local/lib/python3.8/dist-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (52.0.0)
Requirement already satisfied: tensorboard-data-server<0.7.0,>=0.6.0 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (0.6.1)
Requirement already satisfied: tensorboard-plugin-wit>=1.6.0 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (1.8.0)
Requirement already satisfied: markdown>=2.6.8 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (3.3.3)
Requirement already satisfied: rsa<5,>=3.1.4 in /databricks/python3/lib/python3.8/site-packages (from google-auth<2,>=1.6.3->tensorboard~=2.4->tensorflow-cpu==2.4.*) (4.7.2)
Requirement already satisfied: cachetools<5.0,>=2.0.0 in /databricks/python3/lib/python3.

In [0]:
# File location and type
file_location = "/FileStore/tables/Heart-5.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)\
  .drop("Unnamed: 0") #drop unnamed column

display(df)

Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No
56,1,nontypical,120,236,0,0,178,0,0.8,1,0.0,normal,No
62,0,asymptomatic,140,268,0,2,160,0,3.6,3,2.0,normal,Yes
57,0,asymptomatic,120,354,0,0,163,1,0.6,1,0.0,normal,No
63,1,asymptomatic,130,254,0,2,147,0,1.4,2,1.0,reversable,Yes
53,1,asymptomatic,140,203,1,2,155,1,3.1,3,0.0,reversable,Yes


In [0]:
# Create indexer for all the characteristic column
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in list(set(df.columns)-set(['Age',
 'Sex',
 'RestBP',
 'Chol',
 'Fbs',
 'RestECG',
 'MaxHR',
 'ExAng',
 'Oldpeak',
 'Slope',
 'Ca'])) ]
pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(df).transform(df)

df_r.show()

+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+
Age|Sex| ChestPain|RestBP|Chol|Fbs|RestECG|MaxHR|ExAng|Oldpeak|Slope| Ca| Thal|AHD|ChestPain_index|Thal_index|AHD_index|
+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+
 63| 1| typical| 145| 233| 1| 2| 150| 0| 2.3| 3|0.0| fixed| No| 3.0| 2.0| 0.0|
 67| 1|asymptomatic| 160| 286| 0| 2| 108| 1| 1.5| 2|3.0| normal|Yes| 0.0| 0.0| 1.0|
 67| 1|asymptomatic| 120| 229| 0| 2| 129| 1| 2.6| 2|2.0|reversable|Yes| 0.0| 1.0| 1.0|
 37| 1| nonanginal| 130| 250| 0| 0| 187| 0| 3.5| 3|0.0| normal| No| 1.0| 0.0| 0.0|
 41| 0| nontypical| 130| 204| 0| 2| 172| 0| 1.4| 1|0.0| normal| No| 2.0| 0.0| 0.0|
 56| 1| nontypical| 120| 236| 0| 0| 178| 0| 0.8| 1|0.0| normal| No| 2.0| 0.0| 0.0|
 62| 0|asymptomatic| 140| 268| 0| 2| 160| 0| 3.6| 3|2.0| normal|Yes| 0.0| 0.0| 1.0|
 57| 0|asymptomatic| 120| 354| 0| 0| 163| 1| 0.6| 1|0.0| normal| No| 0.0| 0.0| 0.0|
 63| 1|asymptomatic| 130| 254| 0| 2| 147| 0| 1.4| 2|1.0|reversable|Yes| 0.0| 1.0| 1.0|
 53| 1|asymptomatic| 140| 203| 1| 2| 155| 1| 3.1| 3|0.0|reversable|Yes| 0.0| 1.0| 1.0|
 57| 1|asymptomatic| 140| 192| 0| 0| 148| 0| 0.4| 2|0.0| fixed| No| 0.0| 2.0| 0.0|
 56| 0| nontypical| 140| 294| 0| 2| 153| 0| 1.3| 2|0.0| normal| No| 2.0| 0.0| 0.0|
 56| 1| nonanginal| 130| 256| 1| 2| 142| 1| 0.6| 2|1.0| fixed|Yes| 1.0| 2.0| 1.0|
 44| 1| nontypical| 120| 263| 0| 0| 173| 0| 0.0| 1|0.0|reversable| No| 2.0| 1.0| 0.0|
 52| 1| nonanginal| 172| 199| 1| 0| 162| 0| 0.5| 1|0.0|reversable| No| 1.0| 1.0| 0.0|
 57| 1| nonanginal| 150| 168| 0| 0| 174| 0| 1.6| 1|0.0| normal| No| 1.0| 0.0| 0.0|
 48| 1| nontypical| 110| 229| 0| 0| 168| 0| 1.0| 3|0.0|reversable|Yes| 2.0| 1.0| 1.0|
 54| 1|asymptomatic| 140| 239| 0| 0| 160| 0| 1.2| 1|0.0| normal| No| 0.0| 0.0| 0.0|
 48| 0| nonanginal| 130| 275| 0| 0| 139| 0| 0.2| 1|0.0| normal| No| 1.0| 0.0| 0.0|
 49| 1| nontypical| 130| 266| 0| 0| 171| 0| 0.6| 1|0.0| normal| No| 2.0| 0.0| 0.0|
+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+
only showing top 20 rows

In [0]:
# Creat OneHotEncoder except the targe column AHD_index
encoder = OneHotEncoder(inputCols=["ChestPain_index","Thal_index"],
                        outputCols=["ChestPain_Vec", "Thal_Vec"])
target = encoder.fit(df_r)
encoded = target.transform(df_r)
encoded.show()

+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+-------------+-------------+
Age|Sex| ChestPain|RestBP|Chol|Fbs|RestECG|MaxHR|ExAng|Oldpeak|Slope| Ca| Thal|AHD|ChestPain_index|Thal_index|AHD_index|ChestPain_Vec| Thal_Vec|
+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+-------------+-------------+
 63| 1| typical| 145| 233| 1| 2| 150| 0| 2.3| 3|0.0| fixed| No| 3.0| 2.0| 0.0| (3,[],[])| (2,[],[])|
 67| 1|asymptomatic| 160| 286| 0| 2| 108| 1| 1.5| 2|3.0| normal|Yes| 0.0| 0.0| 1.0|(3,[0],[1.0])|(2,[0],[1.0])|
 67| 1|asymptomatic| 120| 229| 0| 2| 129| 1| 2.6| 2|2.0|reversable|Yes| 0.0| 1.0| 1.0|(3,[0],[1.0])|(2,[1],[1.0])|
 37| 1| nonanginal| 130| 250| 0| 0| 187| 0| 3.5| 3|0.0| normal| No| 1.0| 0.0| 0.0|(3,[1],[1.0])|(2,[0],[1.0])|
 41| 0| nontypical| 130| 204| 0| 2| 172| 0| 1.4| 1|0.0| normal| No| 2.0| 0.0| 0.0|(3,[2],[1.0])|(2,[0],[1.0])|
 56| 1| nontypical| 120| 236| 0| 0| 178| 0| 0.8| 1|0.0| normal| No| 2.0| 0.0| 0.0|(3,[2],[1.0])|(2,[0],[1.0])|
 62| 0|asymptomatic| 140| 268| 0| 2| 160| 0| 3.6| 3|2.0| normal|Yes| 0.0| 0.0| 1.0|(3,[0],[1.0])|(2,[0],[1.0])|
 57| 0|asymptomatic| 120| 354| 0| 0| 163| 1| 0.6| 1|0.0| normal| No| 0.0| 0.0| 0.0|(3,[0],[1.0])|(2,[0],[1.0])|
 63| 1|asymptomatic| 130| 254| 0| 2| 147| 0| 1.4| 2|1.0|reversable|Yes| 0.0| 1.0| 1.0|(3,[0],[1.0])|(2,[1],[1.0])|
 53| 1|asymptomatic| 140| 203| 1| 2| 155| 1| 3.1| 3|0.0|reversable|Yes| 0.0| 1.0| 1.0|(3,[0],[1.0])|(2,[1],[1.0])|
 57| 1|asymptomatic| 140| 192| 0| 0| 148| 0| 0.4| 2|0.0| fixed| No| 0.0| 2.0| 0.0|(3,[0],[1.0])| (2,[],[])|
 56| 0| nontypical| 140| 294| 0| 2| 153| 0| 1.3| 2|0.0| normal| No| 2.0| 0.0| 0.0|(3,[2],[1.0])|(2,[0],[1.0])|
 56| 1| nonanginal| 130| 256| 1| 2| 142| 1| 0.6| 2|1.0| fixed|Yes| 1.0| 2.0| 1.0|(3,[1],[1.0])| (2,[],[])|
 44| 1| nontypical| 120| 263| 0| 0| 173| 0| 0.0| 1|0.0|reversable| No| 2.0| 1.0| 0.0|(3,[2],[1.0])|(2,[1],[1.0])|
 52| 1| nonanginal| 172| 199| 1| 0| 162| 0| 0.5| 1|0.0|reversable| No| 1.0| 1.0| 0.0|(3,[1],[1.0])|(2,[1],[1.0])|
 57| 1| nonanginal| 150| 168| 0| 0| 174| 0| 1.6| 1|0.0| normal| No| 1.0| 0.0| 0.0|(3,[1],[1.0])|(2,[0],[1.0])|
 48| 1| nontypical| 110| 229| 0| 0| 168| 0| 1.0| 3|0.0|reversable|Yes| 2.0| 1.0| 1.0|(3,[2],[1.0])|(2,[1],[1.0])|
 54| 1|asymptomatic| 140| 239| 0| 0| 160| 0| 1.2| 1|0.0| normal| No| 0.0| 0.0| 0.0|(3,[0],[1.0])|(2,[0],[1.0])|
 48| 0| nonanginal| 130| 275| 0| 0| 139| 0| 0.2| 1|0.0| normal| No| 1.0| 0.0| 0.0|(3,[1],[1.0])|(2,[0],[1.0])|
 49| 1| nontypical| 130| 266| 0| 0| 171| 0| 0.6| 1|0.0| normal| No| 2.0| 0.0| 0.0|(3,[2],[1.0])|(2,[0],[1.0])|
+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+-------------+-------------+
only showing top 20 rows

In [0]:
# Asseble the OneHoteEcodended orderly
assembler = VectorAssembler(inputCols=['Age', 'Sex','RestBP', 'Chol', 'Fbs','RestECG','MaxHR','ExAng', 'Oldpeak', 'Slope','Ca','AHD_index', 'ChestPain_Vec', 'Thal_Vec'], outputCol='features')
df_enc = assembler.transform(encoded)
df_enc.show(5)

+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+-------------+-------------+--------------------+
Age|Sex| ChestPain|RestBP|Chol|Fbs|RestECG|MaxHR|ExAng|Oldpeak|Slope| Ca| Thal|AHD|ChestPain_index|Thal_index|AHD_index|ChestPain_Vec| Thal_Vec| features|
+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+-------------+-------------+--------------------+
 63| 1| typical| 145| 233| 1| 2| 150| 0| 2.3| 3|0.0| fixed| No| 3.0| 2.0| 0.0| (3,[],[])| (2,[],[])|(17,[0,1,2,3,4,5,...|
 67| 1|asymptomatic| 160| 286| 0| 2| 108| 1| 1.5| 2|3.0| normal|Yes| 0.0| 0.0| 1.0|(3,[0],[1.0])|(2,[0],[1.0])|[67.0,1.0,160.0,2...|
 67| 1|asymptomatic| 120| 229| 0| 2| 129| 1| 2.6| 2|2.0|reversable|Yes| 0.0| 1.0| 1.0|(3,[0],[1.0])|(2,[1],[1.0])|[67.0,1.0,120.0,2...|
 37| 1| nonanginal| 130| 250| 0| 0| 187| 0| 3.5| 3|0.0| normal| No| 1.0| 0.0| 0.0|(3,[1],[1.0])|(2,[0],[1.0])|(17,[0,1,2,3,6,8,...|
 41| 0| nontypical| 130| 204| 0| 2| 172| 0| 1.4| 1|0.0| normal| No| 2.0| 0.0| 0.0|(3,[2],[1.0])|(2,[0],[1.0])|(17,[0,2,3,5,6,8,...|
+---+---+------------+------+----+---+-------+-----+-----+-------+-----+---+----------+---+---------------+----------+---------+-------------+-------------+--------------------+
only showing top 5 rows

In [0]:
#keep in the Dataframe 
df_model=df_enc.select("features","AHD_index")

In [0]:
# Feature need to create modeling
df_model.show()

+--------------------+---------+
 features|AHD_index|
+--------------------+---------+
(17,[0,1,2,3,4,5,...| 0.0|
[67.0,1.0,160.0,2...| 1.0|
[67.0,1.0,120.0,2...| 1.0|
(17,[0,1,2,3,6,8,...| 0.0|
(17,[0,2,3,5,6,8,...| 0.0|
(17,[0,1,2,3,6,8,...| 0.0|
[62.0,0.0,140.0,2...| 1.0|
(17,[0,2,3,6,7,8,...| 0.0|
[63.0,1.0,130.0,2...| 1.0|
[53.0,1.0,140.0,2...| 1.0|
(17,[0,1,2,3,6,8,...| 0.0|
(17,[0,2,3,5,6,8,...| 0.0|
[56.0,1.0,130.0,2...| 1.0|
(17,[0,1,2,3,6,9,...| 0.0|
(17,[0,1,2,3,4,6,...| 0.0|
(17,[0,1,2,3,6,8,...| 0.0|
(17,[0,1,2,3,6,8,...| 1.0|
(17,[0,1,2,3,6,8,...| 0.0|
(17,[0,2,3,6,8,9,...| 0.0|
(17,[0,1,2,3,6,8,...| 0.0|
+--------------------+---------+
only showing top 20 rows

In [0]:
from pyspark.ml.regression import LinearRegression
train, test = df_model.randomSplit([0.8, 0.2], 42)

In [0]:
# Model building
lr = LinearRegression(featuresCol = 'features', labelCol='AHD_index')


In [0]:
#fit with train dataset
lr_model = lr.fit(train)

In [0]:
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-6.094546120599323e-17,-7.49021655660965e-16,-2.008903595040956e-17,-3.3316996352094723e-18,2.521019636354559e-16,9.402697097871936e-17,-3.703065888382609e-17,-1.5124913545455082e-16,3.737454469807497e-17,-6.825410573641365e-16,6.884475469970127e-17,0.9999999999999994,-1.4596240055257682e-15,-1.2744385954105578e-15,-1.1976752563595562e-15,-1.2780452688512276e-15,-6.03996900430349e-16]
Intercept: 1.6224693008055702e-14

In [0]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.000000
r2: 1.000000

In [0]:
lr_predictions = lr_model.transform(test)

In [0]:
predictions = lr_model.transform(test)
predictions.select("prediction","AHD_index","features").show()

+--------------------+---------+--------------------+
 prediction|AHD_index| features|
+--------------------+---------+--------------------+
5.302647111294816...| 0.0|(17,[0,1,2,3,4,5,...|
-3.45746527810968...| 0.0|(17,[0,1,2,3,4,6,...|
-6.67688510750739...| 0.0|(17,[0,1,2,3,4,6,...|
9.33449429670191E-16| 0.0|(17,[0,1,2,3,5,6,...|
-2.48753162209166...| 0.0|(17,[0,1,2,3,5,6,...|
-9.46225669263852...| 0.0|(17,[0,1,2,3,5,6,...|
-1.76702472769719...| 0.0|(17,[0,1,2,3,5,6,...|
-1.29854142039817...| 0.0|(17,[0,1,2,3,5,6,...|
1.585459631698210...| 0.0|(17,[0,1,2,3,6,7,...|
 0.9999999999999999| 1.0|(17,[0,1,2,3,6,7,...|
1.376685232288922...| 0.0|(17,[0,1,2,3,6,7,...|
-1.05142028617444...| 0.0|(17,[0,1,2,3,6,7,...|
 0.9999999999999999| 1.0|(17,[0,1,2,3,6,8,...|
3.454362988982305...| 0.0|(17,[0,1,2,3,6,8,...|
2.593348834544946...| 0.0|(17,[0,1,2,3,6,8,...|
-1.61783963576903...| 0.0|(17,[0,1,2,3,6,8,...|
 0.9999999999999993| 1.0|(17,[0,1,2,3,6,9,...|
3.026372275741903...| 0.0|(17,[0,2,3,4,6,9,...|
-3.13548926342122...| 0.0|(17,[0,2,3,5,6,8,...|
-9.68540439808084...| 0.0|(17,[0,2,3,5,6,8,...|
+--------------------+---------+--------------------+
only showing top 20 rows